# Extraccion de informacion

Script para extraer información desde Metacritic por medio de tecnicas de web scraping. Deja como salida un archivo csv con la información de las reviews

In [22]:
import requests
from bs4 import BeautifulSoup
import time
import random as rand 
import pandas as pd
from langdetect import detect

In [31]:
review_dict = pd.read_csv('./trainingSet.csv')

In [32]:
# Lectura del archivo de urls
resources = pd.read_csv('./urls.csv')
urls = resources.loc[resources['status'] != 'S']

In [48]:
# Lectura del archivo de control
control = pd.read_csv('./control.csv')

In [34]:
# Configuracion de request
headers = {'User-agent': 'Mozilla/5.0'}

In [49]:
i = 1
for url in urls.url:
    print('Procesando url {} de {}'.format(i, len(urls)))
    # Se toma el numero de paginas de reviews
    response = requests.get(url, headers=headers)
    if(response.status_code != 200):
        resources.loc[resources.url == url, 'status'] = 'F'
        continue
    resources.loc[resources.url == url, 'status'] = 'W'
    soup = BeautifulSoup(response.text, 'html.parser')
    lastPageList = soup.find('li', class_='last_page')
    numPages = 10
    if(lastPageList):
        pageRef = lastPageList.find('a', class_='page_num')
        if(pageRef):
            numPages = int(pageRef.text)
    resources.loc[resources.url == url, 'paginasProcesadas'] = 0
    resources.loc[resources.url == url, 'paginasTotales'] = numPages
    for page in range(0,numPages):
        print('  {:.1%}\r'.format((page+1)/numPages),end='')
        filter1 = control['url'] == url
        filter2 = control['page'] == page
        filter3 = control['status'] == 'S'
        register = control.query('url == "'+url+'" and page == '+str(page)+' and status == "S"')
        if(len(register) > 0):
            resources.loc[resources.url == url, 'paginasProcesadas'] += 1
            continue
        headers = {'User-agent': 'Mozilla/5.0'}
        reviewUrl = url + '?page=' + str(numPages)
        response  = requests.get(url, headers = headers)
        if(response.status_code != 200):
            if(len(control.loc[filter1 & filter2]) > 0):
                control.loc[filter1 & filter2,'status'] = 'F'
            else:
                control.loc[len(control)] = {'url':url,'page':page,'status':'F'}
            continue
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.find('div', class_='product_title')
        reviews = soup.find('div', class_='product_reviews')
        product = ''
        platform = ''
        if(title):
            productHead = title.find('h1')
            if(productHead):
                product = productHead.text
            platformHead = title.find('span', class_='platform')
            if(platformHead):
                platform = platformHead.text.strip()
        if(product):
            for review in reviews.find_all('div', class_='review_content'):
                row = {'name':'', 'product':'', 'platform':'', 'date':'', 'rating':'', 'upVotes':'', 'totVotes':'', 'review':'', 'langreview':''}
                name = review.find('div', class_='name')
                if name:
                    row['name'] = name.text.strip().replace('\n','')
                row['product'] = product
                row['platform'] = platform
                date = review.find('div', class_='date')
                if(date):
                   row['date'] = date.text 
                rating = review.find('div', class_='review_grade')
                if(rating):
                    row['rating'] = rating.text.strip().replace('\n','')
                upsVotes = review.find('span', class_='total_ups')
                if(upsVotes):
                    row['upVotes'] = upsVotes.text
                totVotes = review.find('span', class_='total_thumbs')
                if(totVotes):
                    row['totVotes'] = totVotes.text
                content = review.find('div', class_='review_body')
                if(content):
                    row['review'] = content.text.replace('\n',' ').strip()
                else:
                    content = review.find('span', class_='blurb blurb_expanded')
                    if(content):
                        row['review'] = content.text.replace('\n',' ').strip()
                review_dict.loc[len(review_dict)] = row
            if(len(control.loc[filter1 & filter2]) > 0):
                control.loc[filter1 & filter2,'status'] = 'S'
            else:
                control.loc[len(control)] = {'url':url,'page':page,'status':'S'}
            resources.loc[resources.url == url, 'paginasProcesadas'] += 1
    if(resources.loc[resources.url == url, 'paginasProcesadas'][0] == resources.loc[resources.url == url, 'paginasTotales'][0]):
        resources.loc[resources.url == url, 'status'] = 'S'
    else:
        resources.loc[resources.url == url, 'status'] = 'P'
    i += 1

Procesando url 1 de 1
  14.3%  28.6%  42.9%  57.1%  71.4%  85.7%  100.0%

In [36]:
review_dict['langreview'] = review_dict['review'].apply(detect)

In [37]:
control.to_csv('control.csv',index=False)
resources.to_csv('urls.csv',index=False)

In [38]:
review_dict.to_csv('trainingSet.csv', index = False, header=True)